# Annotating plots - Gaia - CMD

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from matplotlib.patches import Rectangle, Circle, Ellipse, Polygon

from astropy.table import QTable
from astropy import units as u

from astroplan import Observer, FixedTarget
from astropy.coordinates import SkyCoord
from astroplan.plots import plot_finder_image

from astroquery.gaia import Gaia

import warnings
warnings.filterwarnings("ignore", category = UserWarning)

### NGC 2682 (M 67) is a very well studied open star cluster in the northern skies

- Right ascension: 08h 51.3m
- Declination: +11° 49′

In [ ]:
my_object_ra = '08h51.3m'
my_object_dec = '+11d49m'

my_object_name = 'NGC 2682'

### Use `FixedTarget` from `astroplan` to get setup a target object.

- #### `FixedTarget(coord = SkyCoord(ra = my_object_ra, dec = my_object_dec), name = NAME)`

In [ ]:
my_target = FixedTarget(coord = SkyCoord(ra = my_object_ra, dec = my_object_dec), name = my_object_name)

In [ ]:
my_target.coord

In [ ]:
my_target.dec.degree

In [ ]:
my_target.ra.degree

In [ ]:
my_target.ra.hms

#### We can use `plot_finder_image` to get an image of the object from the [ESO Online Digitized Sky Survey](http://archive.eso.org/dss/dss).

In [ ]:
fig = plt.subplots(
    figsize = (6, 6), 
    constrained_layout = True
)

ax, hdu = plot_finder_image(my_target, fov_radius= 0.4 * u.deg);

## Get the the [Gaia data](https://www.cosmos.esa.int/web/gaia/dr3) for the object

- Note the use of `f-string` and variables in the query
- Gaia wants the object coordinates in degrees

In [ ]:
my_query = f"""
SELECT TOP 3000
source_id, ra, dec, phot_g_mean_mag, bp_rp, parallax
FROM gaiadr3.gaia_source_lite
WHERE CONTAINS(
   POINT('ICRS', {my_target.ra.degree}, {my_target.dec.degree}),
   CIRCLE('ICRS', ra, dec, 0.3)
   ) = 1
AND parallax > 0.1
AND bp_rp IS NOT NULL
ORDER BY parallax DESC
"""

In [ ]:
print(my_query)

In [ ]:
my_job_query = Gaia.launch_job(my_query)

In [ ]:
print(my_job_query)

In [ ]:
my_table = my_job_query.get_results()

In [ ]:
my_table[0:2]

### Use `parallax` and `phot_g_mean_mag` to get **distance** and **absolute magnitude**

- Add then as columns to the data table

In [ ]:
my_table['distance'] = my_table['parallax'].to(u.parsec, equivalencies=u.parallax())

In [ ]:
my_table[0:2]

In [ ]:
def find_absmag(my_gmag, my_distance):
    result = my_gmag - 5 * np.log10( my_distance / (10 * u.parsec)) * u.mag
    return result

In [ ]:
my_table['abs_g'] = find_absmag(my_table['phot_g_mean_mag'], my_table['distance']) * u.mag

In [ ]:
my_table[0:2]

## Plot a histogram of the distances

- All the stars in the cluster should have the same distance
- Objects not in the cluster will have different distances

In [ ]:
fig, ax = plt.subplot_mosaic(
    '''
    AB
    ''',
    figsize = (12, 4), 
    constrained_layout = True
)

ax['A'].set_xlabel("Distance (pc)")
ax['A'].set_ylabel("Number")

ax['A'].hist(my_table['distance'],
        bins = 100,
        histtype = 'stepfilled',
        facecolor = 'MediumOrchid')

ax['B'].set_xlim(0, 2000)

ax['B'].set_xlabel("Distance (pc)")
ax['B'].set_ylabel("Number")

ax['B'].hist(my_table['distance'],
        bins = 100,
        histtype = 'stepfilled',
        facecolor = 'MediumOrchid');

## Pretty easy to see the cluster's distance

- We will use distances between 750 pc and 1000 pc for cluster stars
- The cluster distance from the literature is 800 - 900 pc, so these values seems fine.

In [ ]:
my_cluster_table = my_table[(my_table['distance'] > 750) &
                            (my_table['distance'] < 1000)
                           ]

In [ ]:
len(my_cluster_table) / len(my_table)

## Make a color magnitude diagram (CMD) of the object


Color Magnitude Diagram (CMD) is a plot of Color Index vs. Magnitude. This is just a HR-diagram with a change of units.

Some things we have to keep in mind when making a CMD

- Color Index (X-axis) cover a very small range of values, we will need to adjust our axes accordingly.
- Magnitudes (Y-axis) are backwards, we will need to adjust our axes accordingly.

#### `np.ptp()` returns the range of values (max - min) for an array (*P*eak *T*o *P*eak)

In [ ]:
np.ptp(my_cluster_table['bp_rp'])

In [ ]:
np.ptp(my_cluster_table['abs_g'])

In [ ]:
fig, ax = plt.subplots(
    figsize = (15, 15), 
    constrained_layout = True
)

# Color Index cover a very small range of values
# Make 1 unit in X = 4 units in Y

ax.set_aspect(1 / 4)

# Magnitudes are backwards

ax.set_ylim(-2.5,13)
ax.invert_yaxis()

###

ax.set_xlabel("BP - RP",
              fontfamily = 'serif',
              fontsize = 25)

ax.set_ylabel(r"$G_{M}$",
              fontfamily = 'serif',
              fontsize = 25)

ax.set_title(f"{my_target.name}",
             fontfamily = 'serif',
             fontsize = 30)

### Plot Data ###

ax.plot(my_cluster_table['bp_rp'], my_cluster_table['abs_g'],
        color = "#4C0B5F",
        marker = "o",
        linestyle = "None",
        markersize = 5
       );

---

# Annotate your plots with `annotate()`

### There are 4 parameters that you use with `annotate()`.

- `text` : the text label (string).
- `xy = (X, Y)` : The coordinates (X, Y) where you want your arrowhead to point to.
- `xytext = (X, Y)` : The coordinates (X, Y) where you want your `text` to display.
- `arrowprops` : A dictionary of key-value pairs which define various properties for the arrow, such as color, size and arrowhead type.
  - [List of Matplotlib Arrowstyles](https://coderslegacy.com/python/matplotlib-customize-arrowstyles/)
  - [List of Matplotlib Arrowprops](https://coderslegacy.com/python/matplotlib-arrowprops/)
  
---

In [ ]:
fig, ax = plt.subplots(
    figsize = (15, 15), 
    constrained_layout = True
)

# Color Index cover a very small range of values
# Make 1 unit in X = 4 units in Y

ax.set_aspect(1/4)

# Magnitudes are backwards

ax.set_ylim(-2.5,13)
ax.invert_yaxis()

###

ax.set_xlabel("BP - RP",
              fontfamily = 'serif',
              fontsize = 25)

ax.set_ylabel(r"$G_{M}$",
              fontfamily = 'serif',
              fontsize = 25)

ax.set_title(f"{my_target.name}",
             fontfamily = 'serif',
             fontsize = 30)

### Plot Data ###

ax.plot(my_cluster_table['bp_rp'], my_cluster_table['abs_g'],
        color = "#4C0B5F",
        marker = "o",
        linestyle = "None",
        markersize = 5)

# Sun (BP-RP = 0.82,  G_M = 4.67)
        
ax.plot(0.82, 4.67,
        color = "DarkOrange",
        marker = "*",
        linestyle = "None",
        markersize = 30
       )

### Text Annotation ###

ax.annotate('Main Sequence',
             fontsize = 25,
             color = 'green',
             xy = (2.0, 7.0),
             xytext = (2.0, 5.0),
             arrowprops = {'color' : 'green',
                           'linewidth' : 4,
                           'arrowstyle' : '->, head_length = 0.8, head_width = 0.5'}
           )

ax.annotate('Red Giant\nBranch',
             fontsize = 25,
             color = 'red',
             xy = (1.3, 1.5),
             xytext = (1.7, 0.0),
             arrowprops = {'color' : 'red',
                           'linewidth' : 4,
                           'arrowstyle' : '-|>, head_length = 0.8, head_width = 0.3',
                           'connectionstyle' : 'angle3'}
           )

ax.annotate('Sun',
             fontsize = 25,
             color = 'DarkOrange',
             xy = (0.75, 4.67),
             xytext = (0.3, 6.0),
             arrowprops = {'color' : 'DarkOrange',
                           'linewidth' : 4,
                           'arrowstyle' : '-|>, head_length = 0.8, head_width = 0.3',
                           'connectionstyle' : 'angle3'}
           );

---

# Annotate your plots with Shapes


### [Drawing Shapes with Matplotlib Patches](https://coderslegacy.com/python/drawing-shapes-matplotlib-patches/)
---

### `Ellipse((center), width, height, angle=0.0, **kwargs)`

### `Rectangle((x,y), width, height, angle=0.0, rotation_point='xy', **kwargs)`

In [ ]:
fig, ax = plt.subplots(
    figsize = (15, 15), 
    constrained_layout = True
)

# Color Index cover a very small range of values
# Make 1 unit in X = 4 units in Y

ax.set_aspect(1/4)

# Magnitudes are backwards

ax.set_ylim(-2.5,13)
ax.invert_yaxis()

###

ax.set_xlabel("BP - RP",
              fontfamily = 'serif',
              fontsize = 25)

ax.set_ylabel(r"$G_{M}$",
              fontfamily = 'serif',
              fontsize = 25)

ax.set_title(f"{my_target.name}",
             fontfamily = 'serif',
             fontsize = 30)

### Plot Data ###

ax.plot(my_cluster_table['bp_rp'], my_cluster_table['abs_g'],
        color = "#4C0B5F",
        marker = "o",
        linestyle = "None",
        markersize = 5)

# Sun (BP-RP = 0.82,  G_M = 4.67)
        
ax.plot(0.82, 4.67,
        color = "DarkOrange",
        marker = "*",
        linestyle = "None",
        markersize = 30
       )

### Text Annotation ###

ax.annotate('Main Sequence',
             fontsize = 25,
             color = 'green',
             xy = (2.0, 7.0),
             xytext = (2.0, 5.0),
             arrowprops = {'color' : 'green',
                           'linewidth' : 4,
                           'arrowstyle' : '->, head_length = 0.8, head_width = 0.5'}
           )

ax.annotate('Red Giant\nBranch',
             fontsize = 25,
             color = 'red',
             xy = (1.3, 1.5),
             xytext = (1.7, 0.0),
             arrowprops = {'color' : 'red',
                           'linewidth' : 4,
                           'arrowstyle' : '-|>, head_length = 0.8, head_width = 0.3',
                           'connectionstyle' : 'angle3'}
           )

ax.annotate('Sun',
             fontsize = 25,
             color = 'DarkOrange',
             xy = (0.75, 4.67),
             xytext = (0.3, 6.0),
             arrowprops = {'color' : 'DarkOrange',
                           'linewidth' : 4,
                           'arrowstyle' : '-|>, head_length = 0.8, head_width = 0.3',
                           'connectionstyle' : 'angle3'}
           )

### Shape Annotation ###

my_shape_one = Ellipse((0.35, 2.0),
                       width = 0.75,
                       height = 2.85,
                       angle = 0,
                       color = (0, 0, 0.7, 0.2))

ax.add_patch(my_shape_one)

ax.text(-0.1, 0.4,
       'Blue Stragglers',
        color = (0, 0, 0.7, 0.5),
        fontsize = 24)

my_shape_two = Rectangle((-0.6, 13.0),
                       width = 1.5,
                       height = -3.0,
                       angle = 0,
                       color = (0.7, 0, 0.0, 0.5))

ax.add_patch(my_shape_two)

ax.text(-0.5, 12.5,
       'White Dwarfs(?)',
        color = (0.7, 0, 0.0),
        fontsize = 24);